In [6]:
!pip install imutils
!pip install imutils
!pip install dlib
!pip install pygame

In [2]:
import numpy as np
import imutils
import time
import timeit
import dlib
import cv2
import matplotlib.pyplot as plt
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
from threading import Timer

In [4]:
import time

def check_fps(prev_time) :
    cur_time = time.time() #Import the current time in seconds
    one_loop_time = cur_time - prev_time
    prev_time = cur_time
    fps = 1/one_loop_time
    return prev_time, fps

In [5]:
def check_fps(prev_time) :
    cur_time = time.time() #Import the current time in seconds
    one_loop_time = cur_time - prev_time
    prev_time = cur_time
    fps = 1/one_loop_time
    return prev_time, fps
def light_removing(frame) :
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    L = lab[:,:,0]
    med_L = cv2.medianBlur(L,99) #median filter
    invert_L = cv2.bitwise_not(med_L) #invert lightness
    composed = cv2.addWeighted(gray, 0.75, invert_L, 0.25, 0)
    return L, composed



In [6]:
knn = cv2.ml.KNearest_create()
knn.load('knn_model.xml')  # Load the model

< cv2.ml.KNearest 000001CD99982770>

In [9]:
import mysql.connector
import sqlite3
import datetime
import time
import mysql.connector
from datetime import datetime

In [11]:
# 졸음 2진분류 / 졸음 시간 노멀시간 싸이클로 계산
# 데이터베이스 연결 설정
db = mysql.connector.connect(
    host="project-db-campus.smhrd.com",
    port=3312,
    user="seocho_22K_DCX_final_2",
    password="smhrd2",
    database="seocho_22K_DCX_final_2"
)
cursor = db.cursor()

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    eye = (A + B) / (2.0 * C)
    return eye

def init_open_eye():
    time.sleep(5)
    print("open init time sleep")
    eye_list = []
    for i in range(7):
        eye_list.append(both_eye)
        time.sleep(1)
    global OPEN_eye
    OPEN_eye = sum(eye_list) / len(eye_list)
    print("OPEN_eye =", OPEN_eye)

def init_close_eye():
    time.sleep(2)
    th_open.join()
    time.sleep(5)
    print("close init time sleep")
    eye_list = []
    time.sleep(1)
    for i in range(7):
        eye_list.append(both_eye)
        time.sleep(1)
    CLOSE_eye = sum(eye_list) / len(eye_list)
    global eye_THRESH
    eye_THRESH = (((OPEN_eye - CLOSE_eye) / 2) + CLOSE_eye) 
    print("CLOSE_eye =", CLOSE_eye)
    print("The last eye_THRESH's value :",eye_THRESH)

OPEN_eye = 0
eye_THRESH = 0.3
eye_CONSEC_FRAMES = 10
COUNTER = 0

closed_eyes_time = []
TIMER_FLAG = False
ALARM_FLAG = False

ALARM_COUNT = 0
RUNNING_TIME = 0
PREV_TERM = 0

np.random.seed(30)

test_data = []
result_data = []
prev_time = 0
print("loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
print("starting video stream thread...")
vs = VideoStream(src=0).start()
time.sleep(1.0)
th_open = Thread(target = init_open_eye)
th_open.deamon = True
th_open.start()
th_close = Thread(target = init_close_eye)
th_close.deamon = True
th_close.start()

start_normal = None  # 노말타임을 측정하기 위한 시작 시간 변수 추가

while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    L, gray = lr.light_removing(frame)
    rects = detector(gray, 0)
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        both_eye = (leftEAR + rightEAR) * 500
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        if both_eye < eye_THRESH:
            COUNTER += 1

            if COUNTER >= eye_CONSEC_FRAMES:
                if not TIMER_FLAG:
                    start_closing = timeit.default_timer()
                    if start_normal:  # 노말타임 측정
                        end_normal = timeit.default_timer()
                        normal_duration = end_normal - start_normal
                        start_normal = None  # 노말타임 측정 초기화
                    TIMER_FLAG = True
                cv2.putText(frame, "Status : Sleep", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            if TIMER_FLAG:
                end_closing = timeit.default_timer()
                sleep_duration = end_closing - start_closing
                # 5초 이상 잠잔 경우에만 DB에 잠잔 시간(초)와 노말타임(초) 저장
                if sleep_duration >= 5 and normal_duration:
                    query = "INSERT INTO t_user_sleep(sleep_time, nomal_time) VALUES (%s, %s)"
                    val = (sleep_duration, normal_duration)
                    cursor.execute(query, val)
                    db.commit()
                TIMER_FLAG = False
                start_normal = timeit.default_timer()  # 노말타임 측정 시작
            COUNTER = 0
            cv2.putText(frame, "Status : Normal", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, "eye : {:.2f}".format(both_eye), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 30, 20), 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cv2.destroyAllWindows()
vs.stop()


loading facial landmark predictor...


RuntimeError: Unable to open shape_predictor_68_face_landmarks.dat